In [ ]:
import colorsys
import math
import os
import shutil

import anndata
import distinctipy
import matplotlib
import numpy as np
import pandas as pd
import scanpy as sc
from sklearn import preprocessing

In [ ]:
save_folder = "../input/"

hbc_path = "../../Data/Embryo 1/hbc_mut_embryo1.txt"
mito_path = "../../Data/Embryo 1/mito_mut_embryo1_0.6.txt"
cell_map_type_path = "../../Data/Embryo 1/annotation_main_new3.csv"
rna_path = "../../Data/Embryo 1/concat.h5ad"

# pre process hbc mito

In [ ]:
# input hbc and mito file formats
# cells * mutations

In [ ]:
# hbc of embryo1
hbc = pd.read_csv(hbc_path, header=0, sep="\t", index_col=0)
# regex = ".*-7938-3|.*-7995-1"
# hbc = hbc[hbc.index.str.match(regex)]

# remove mutations present only in <=1 cells. Remove cells which do not contain mutation
hbc = hbc.loc[:, hbc.sum() > 1]
hbc = hbc[hbc.sum(axis=1) > 0]
hbc.to_csv(save_folder + "hbc_mut.txt", sep="\t", index=True)

In [ ]:
# mito of embyro1
mito = pd.read_csv(mito_path, header=0, sep="\t", index_col=0)
# regex = ".*-7938-3|.*-7995-1"
# mito = mito[mito.index.str.match(regex)]

# remove mutations present only in <=1 cells. Remove cells which do not contain mutation
mito = mito.loc[:, mito.sum() > 1]
mito = mito[mito.sum(axis=1) > 0]

In [ ]:
# choose the threshold 0.6 for embryo1

(mito.sum() / mito.shape[0] * 100).sort_values().plot.bar(
    figsize=(20, 8),
    title="Mtio embryo1 mutions vs percentage of cells" + str(mito.shape),
    xlabel="mutaion name",
    ylabel="% of cells",
)

In [ ]:
threshold = 0.6  # need to be selected

In [ ]:
mito_threshold = mito.loc[:, (mito.sum() / mito.shape[0] * 100) < threshold]
mito_threshold = mito_threshold.loc[mito_threshold.sum(axis=1) > 0, :]
mito_threshold

mito_threshold.to_csv(save_folder + "mito_mut.txt", sep="\t", index=True)

# RNA process

In [ ]:
# process RNA as required for lintimat

In [ ]:
adata = sc.read_h5ad(rna_path)
adata_sce = adata.copy()

In [ ]:
import anndata2ri

anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
%%R -i adata_sce -o X_imp -o cell_names -o gene_names
library(Seurat)
library(DrImpute)
library(SummarizedExperiment)

X <- assays(adata_sce)$X
cell_names = colnames(X)
X <- preprocessSC(X, min.expressed.gene = 0)
X.log <- log(X + 1)
set.seed(1)
X_imp <- DrImpute(X.log)

gene_names = rownames(X_imp)

In [ ]:
adata = adata[list(cell_names), list(gene_names)]
adata.X = X_imp.T
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.write(save_folder + "concat_drimpute.h5ad")
adata

In [ ]:
rna = pd.DataFrame(adata.X, columns=adata.var_names, index=adata.obs_names)
top_2000 = pd.DataFrame(rna.var().sort_values(ascending=False)[:2000].index)
top_2000.to_csv(save_folder + "top2000.txt", index=False, header=False)
top_2000

In [ ]:
del adata

# cell type color mapping

In [ ]:
cell_map_type = pd.read_csv(cell_map_type_path, sep=",", index_col=0)
cell_types = sorted(list(set(cell_map_type["ClusterIdent"])))
print("number of cell types ", len(cell_types))

In [ ]:
# ref https://www.alanzucconi.com/2015/09/30/colour-sorting/
colors = distinctipy.get_colors(len(cell_types), rng=0)
distinctipy.color_swatch(colors, one_row=True)
# sort the colors
def step(r, g, b, repetitions=1):
    lum = math.sqrt(0.241 * r + 0.691 * g + 0.068 * b)
    h, s, v = colorsys.rgb_to_hsv(r, g, b)
    h2 = int(h * repetitions)
    lum2 = int(lum * repetitions)
    v2 = int(v * repetitions)

    if h2 % 2 == 1:
        v2 = repetitions - v2
        lum = repetitions - lum

    return (h2, lum, v2)


colors.sort(key=lambda x: step(x[0], x[1], x[2], 8))
print("sorted colors")
distinctipy.color_swatch(colors, one_row=True)

cell_map_color = pd.DataFrame(
    list(zip(sorted(cell_types), colors)), columns=["cluster_type", "color"]
)
cell_map_color = cell_map_color.set_index("cluster_type")
cell_map_color["color"] = cell_map_color["color"].apply(matplotlib.colors.rgb2hex)
cell_map_color.color = cell_map_color.color.apply(lambda x: x.replace("#", ""))
pd.DataFrame(cell_map_color).reset_index().to_csv(
    save_folder + "celltype_map_color.txt", sep="\t", header=True, index=True
)

# Create Mutation profile:  Homing+Mito

zero padded if any one of homing or mito is missing. This mutation profile is generated only for the cells common with rna

In [ ]:
# adata_rna = sc.read_h5ad(rna_path)
annotations = pd.read_csv(cell_map_type_path, sep=",", index_col=0)
print("number of cells", len(set(annotations.index)))
print("number of cell types", len(set(annotations["ClusterIdent"])))

In [ ]:
mito = pd.read_csv("../input/mito_mut.txt", sep="\t", index_col=0)
mito = mito.loc[list(set(mito.index) & set(annotations.index))]
mito.shape

In [ ]:
hbc = pd.read_csv("../input/hbc_mut.txt", index_col=0, sep="\t")
hbc = hbc.loc[list(set(hbc.index) & set(annotations.index))]
hbc.shape

In [ ]:
print("homing and mitochondrial barcodes", len(set(set(hbc.index) | set(mito.index))))

In [ ]:
mutations = pd.concat([hbc, mito], axis=1, join="outer")  # ,ignore_index=True)
mutations = mutations.fillna(0)
# mutations = mutations.astype(int)
mutations = mutations.loc[:, mutations.sum() > 0]
mutations = mutations.loc[
    mutations.sum(axis=1) > 0,
]
mutations.shape

In [ ]:
mutations_meta = annotations.loc[mutations.index]
mutations_meta["annotation"] = annotations["ClusterIdent"].loc[mutations.index]
mutations_meta["mutation_type"] = "hbc"
mutations_meta.loc[mito.index, "mutation_type"] = "mito"
mutations_meta.loc[set(mito.index) & set(hbc.index), "mutation_type"] = "hbc+mito"

# clustering the mutation profiles

In [ ]:
adata_all = anndata.AnnData(mutations.astype(np.float32))
adata_all.obs = mutations_meta
sc.pp.pca(adata_all)
sc.pp.neighbors(adata_all)
sc.tl.umap(adata_all)
for i in [0.01, 0.02, 0.03, 0.07, 0.1]:  # can have different resolutions
    print("resolution : ", i)
    sc.tl.leiden(adata_all, resolution=i)
    sc.pl.umap(adata_all, color="leiden")
# sc.pl.umap(adata,color='annotation')

In [ ]:
sc.pl.umap(adata_all, color="mutation_type")

In [ ]:
# select the resolution that makes sense
res = 0.01
sc.tl.leiden(adata_all, resolution=res)
print("number of cells ", adata_all.shape[0])
print("cluster    number of cells")
print(adata_all.obs["leiden"].value_counts())
sc.pl.umap(adata_all, color="leiden")

In [ ]:
adata_all.write("../input/hbc_mito.h5ad")

In [ ]:
# adata = sc.read_h5ad("../input/hbc_mito.h5ad")

In [ ]:
# print(np.unique(adata.obs["mutation_type"], return_counts=True))
# mutation_count_in_cluster = pd.DataFrame([], columns=["hbc", "hbc+mito", "mito"])
# mutation_count_in_cluster.index.name = "cluster_no"

# for clus in sorted(np.unique(adata.obs["leiden"]).astype(int)):
#     cluster_data = adata.obs[adata.obs["leiden"] == str(clus)]
#     # print('cluster ',clus)
#     temp = pd.Series(np.unique(cluster_data["mutation_type"], return_counts=True))
#     mutation_count_in_cluster.loc[clus, temp[0]] = temp[1]
#     # print(pd.DataFrame(np.unique(cluster_data['mutation_type'],return_counts = True)))

# mutation_color_map_dic = {"mito": "red", "hbc": "green", "hbc+mito": "blue"}
# mutation_count_in_cluster.plot(kind="bar", stacked=True, color=mutation_color_map_dic)

# Create Dataset

In [ ]:
top2000_path = save_folder + "top2000.txt"
celltype_map_color_path = save_folder + "celltype_map_color.txt"
drimpute_rna_path = save_folder + "concat_drimpute.h5ad"

In [ ]:
adata_rna = sc.read_h5ad(drimpute_rna_path)
clusters = sorted(set(adata_all.obs["leiden"].unique()))

In [ ]:
cwd = os.getcwd() + "/"
cwd

In [ ]:
for cluster in clusters:
    print("creating input for cluster : {} ..".format(cluster))
    # create directories
    if not os.path.exists(f"{cwd}../sub_clusters/{cluster}/input/"):
        os.makedirs(f"{cwd}../sub_clusters/{cluster}/input/")
        os.makedirs(f"{cwd}../sub_clusters/{cluster}/output/")

    # get mutations
    adata = adata_all[adata_all.obs["leiden"] == cluster]
    mutations = pd.DataFrame(
        adata.X, index=adata.obs_names, columns=adata.var_names
    ).astype(int)

    mutations = mutations.loc[:, mutations.sum() > 1]
    mutations = mutations.loc[mutations.sum(axis=1) > 0, :]

    mutations = mutations.T
    mutations = mutations.astype(int)
    mutations = mutations.astype(str)

    # get RNA
    adata_rna_sub = adata_rna[mutations.columns, :]

    # get annotations
    # annotation = pd.read_csv(cell_map_type_path,sep='\t',index_col = 0).loc[adata_rna_sub.obs_names]
    # annotation.rename(columns = {'annotation':'ClusterIdent'}, inplace = True)

    # mutation mapping
    le = preprocessing.LabelEncoder()
    le.fit(list(mutations.index))
    le_name_mapping = list(zip(le.classes_, le.transform(le.classes_)))
    pd.DataFrame(le_name_mapping).to_csv(
        f"{cwd}../sub_clusters/{cluster}/input/mutation_mapping.txt",
        sep="\t",
        header=False,
        index=False,
    )

    # cell - mutation string
    for i in range(mutations.shape[0]):
        for j in range(mutations.shape[1]):
            if mutations.iloc[i][j] == "0":
                mutations.iloc[i][j] = "NONE"
            elif mutations.iloc[i][j] == "1":
                mutations.iloc[i][j] = le.transform([mutations.index[i]])[0]
            else:
                print(mutations.iloc[i][j], "error")

    mutations = mutations.astype(str).apply("-".join, axis=0)
    mutations = pd.DataFrame(mutations, columns=["HMID"])

    mutations.to_csv(
        f"{cwd}../sub_clusters/{cluster}/input/cell_mutation_mapping.txt",
        sep="\t",
        header=False,
        index=True,
    )

    # create litimat input format
    rna_data = pd.DataFrame(
        adata_rna_sub.X, index=adata_rna_sub.obs_names, columns=adata_rna_sub.var_names
    )
    lintimat_format = pd.concat(
        [annotations, mutations, rna_data], axis=1, join="inner"
    )
    clusters_to_numbers = pd.DataFrame(
        enumerate(lintimat_format["ClusterIdent"].astype("category").cat.categories)
    )
    lintimat_format["ClusterIdent"] = (
        lintimat_format["ClusterIdent"].astype("category").cat.codes
    )

    lintimat_format.to_csv(
        f"{cwd}../sub_clusters/{cluster}/input/Data_matrix_Comb_final2_for_lintimat.txt",
        sep="\t",
    )
    clusters_to_numbers.to_csv(
        f"{cwd}../sub_clusters/{cluster}/input/lintimat_cell_type_map.txt",
        sep="\t",
        index=False,
        header=False,
    )
    lintimat_format.iloc[:, :2].to_csv(
        f"{cwd}../sub_clusters/{cluster}/input/lintimat_txt_label_HMID.txt", sep="\t"
    )
    shutil.copy2(top2000_path, f"{cwd}../sub_clusters/{cluster}/input/")
    shutil.copy2(celltype_map_color_path, f"{cwd}../sub_clusters/{cluster}/input/")

    del adata
    del adata_rna_sub

In [ ]:
path = "../sub_clusters/"
for cluster in clusters:
    print("running lintimat for {}".format(cluster))
    os.system("date >  " + path + cluster + "/output/terminal_output.txt")
    os.system(
        "java -jar ../code/LinTIMaT.jar -i " + path + cluster + "/input/Data_matrix_Comb_final2_for_lintimat.txt\
        -gf " + path  + cluster  + "/input/top2000.txt \
        -gc 2000 \
        -ob " + path + cluster  + "/output/bin_tree.newick \
        -on " + path + cluster+ "/output/nonbinary_tree.txt \
         -mi 200000 -ci 0 -s 9126 >> "+ path+ cluster+ "/output/terminal_output.txt"
    )

    os.system("date >> " + path + cluster + "/output/terminal_output.txt")